### Wesley Janson and Drew Keller
## STAT 27420 Final Project
# Modeling Code

In [8]:
# Load in relevant packages

import pandas as pd
from statsmodels.miscmodels.ordinal_model import OrderedModel
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from data_utils import prep_features
import numpy as np

In [3]:
from load_data import data, categorical_vars, cts_vars, other_vars
# categorical_vars and cts_vars are lists of vars in each category
# other_vars are ID and date variables (categorical_vars + cts_vars + other_vars = all vars)

/Users/drewkeller/opt/anaconda3/envs/stats/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [4]:
# loading data from online takes ~20 seconds
# to speed up, save data locally and load from there:
DATA_PATH = '../paper_replication_data/new_data.csv'
data.to_csv(DATA_PATH)  # run this once

In [7]:
data = pd.read_csv(DATA_PATH)

NameError: name 'DATA_PATH' is not defined

In [9]:
# prep features for modeling; use regression=True for regression models
data_regression, treatment_vars, confounder_vars = prep_features(data,regression=True)  

In [10]:
treatment_vars

['treatment_bins_10-15',
 'treatment_bins_15-20',
 'treatment_bins_20+',
 'treatment_bins_5-10']

In [5]:
data.treatment_bins.value_counts()  # check that we have a balanced dataset

0-5      163561
5-10      47431
20+       30464
10-15     11780
15-20      4984
Name: treatment_bins, dtype: int64

In [6]:
treatment_bins = ['treatment_bins_5-10','treatment_bins_10-15','treatment_bins_15-20','treatment_bins_20+']

In [7]:
data.durable_purchase.value_counts()  # check that we have a balanced dataset

 1.0    205880
-1.0     72235
 0.0     13212
Name: durable_purchase, dtype: int64

In [8]:
data_regression.durable_purchase.value_counts(dropna=False)

 1.0    205880
-1.0     72235
 0.0     13212
Name: durable_purchase, dtype: int64

In [9]:
# First model, ordered probit-same as Bachmann et al.
mod_prob = OrderedModel(data_regression['durable_purchase'],
                        data_regression[confounder_vars+treatment_vars],
                        distr='probit')

res_prob = mod_prob.fit(method='bfgs')
res_prob.summary()

predicted = res_prob.model.predict(res_prob.params, exog=data_regression[confounder_vars+treatment_bins])
predicted

KeyError: "['sex', 'education', 'income_quintile', 'price_related_yr_ago', 'conditions_next_yr', 'unemployment_next_yr', 'personal_finances_next_yr'] not in index"

In [17]:
res_prob.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:       durable_purchase   Log-Likelihood:            -2.0932e+05
Model:                   OrderedModel   AIC:                         4.187e+05
Method:            Maximum Likelihood   BIC:                         4.187e+05
Date:                Wed, 30 Nov 2022                                         
Time:                        13:46:38                                         
No. Observations:              291327                                         
Df Residuals:                  291319                                         
Df Model:                           8                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
fed_funds_rate           0.0240      0.002      9.825      0.000       0.019       0.029
unemployment_rate       -0.1541      0.002    -63.539      0.000      -0.159      -0.149
treatment_bins_5-10     -0.2108      0.007    -31.149      0.000      -0.224      -0.198
treatment_bins_10-15    -0.3149      0.012    -25.610      0.000      -0.339      -0.291
treatment_bins_15-20    -0.3912      0.018    -21.300      0.000      -0.427      -0.355
treatment_bins_20+      -0.3815      0.008    -47.063      0.000      -0.397      -0.366
-1.0/0.0                -0.7819      0.003   -252.625      0.000      -0.788      -0.776
0.0/1.0                 -1.9640      0.008   -231.696      0.000      -1.981      -1.947
========================================================================================
"""

In [ ]:
# Second model - some sort of XGBoost?